In [1]:
import sys 
sys.path.append('../')

In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
from sampler.sampler_perturbations import sampler_perturbations
from all_tests import mmdfuse_test, mmd_median_test, mmdagg_test, mmdagginc_test


import numpy as np
import jax 
import jax.numpy as jnp
from jax import random 

import itertools
from tqdm.auto import tqdm 
from pathlib import Path 
Path("../results").mkdir(exist_ok=True)

%load_ext autoreload
%autoreload 2

In [4]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


#### Unequal Sample Size

In [5]:
import itertools
import jax.numpy as jnp
from jax import random

# Your existing parameters
repetitions = 200
number_perturbations = 2
scales = [0.3, 0.5]
sample_sizes = [5000, 10000]
s_num = len(scales)
ss_num = len(sample_sizes)
d = 1
tests = (mmdfuse_test, mmd_median_test, mmdagg_test)

# Initializing the arrays
outputs = jnp.zeros((len(tests), len(scales), len(sample_sizes), repetitions))
scale_sample_averages = jnp.zeros((len(scales), len(sample_sizes)))  # New array for averages
key = random.PRNGKey(42)
seed = 42

# Running the tests
for s, ss in tqdm(itertools.product(range(s_num), range(ss_num))):
    m = sample_sizes[ss]
    # n = sample_sizes[0]
    n = 500
    scale = scales[s]
    for i in tqdm(range(repetitions)):
        key, subkey = random.split(key)
        X, Y = sampler_perturbations(m, n, d, number_perturbations, seed, scale)
        key, subkey = random.split(key)
        seed += 1
        for t in range(len(tests)):
            test = tests[t]
            test_result = test(X, Y, subkey, seed)
            outputs = outputs.at[t, s, ss, i].set(test_result)

# Compute the mean across all tests for each scale and sample size
for s, ss in itertools.product(range(s_num), range(ss_num)):
    scale_sample_averages[s][ss] = jnp.mean(outputs[:, s, ss, :])

# Saving and printing the results
jnp.save("../results/perturbations_vary_dif_mn_d1_averages.npy", scale_sample_averages)

print("Averages by Scale and Sample Size")
print("Scales: ", scales)
print("Sample Sizes: ", sample_sizes)
print(scale_sample_averages)

0it [00:00, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]